In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np

In [2]:
from scipy.optimize import minimize

# importing data

## WDI

In [3]:
WDI_df = pd.read_excel(r"cobb_doulgass_calculation/cobb_doulgass_wdi_data.xlsx")

## Penn World Table

In [4]:
penn_table_depreciation_df = pd.read_excel(r"cobb_doulgass_calculation/cobb_doulgass_penn_table_depreciation.xlsx")

# Capital stock

In [5]:
GFCF_df = WDI_df[WDI_df['Indicator Code']=='NE.GDI.FTOT.CD']

In [6]:
GFCF_df

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value,Source
162,China,CHN,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,1960,1.949517e+10,World Bank 2025 06
163,India,IND,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,1960,5.353005e+09,World Bank 2025 06
164,China,CHN,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,1961,9.506394e+09,World Bank 2025 06
165,India,IND,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,1961,6.181446e+09,World Bank 2025 06
166,China,CHN,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,1962,7.307599e+09,World Bank 2025 06
...,...,...,...,...,...,...,...
330,India,IND,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,2021,9.368164e+11,World Bank 2025 06
333,China,CHN,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,2022,7.493288e+12,World Bank 2025 06
336,India,IND,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,2022,1.031188e+12,World Bank 2025 06
339,China,CHN,Gross fixed capital formation (current US$),NE.GDI.FTOT.CD,2023,7.356189e+12,World Bank 2025 06


## at 1978

In [ ]:
GFCF_1960_1978_df = GFCF_df[GFCF_df['Year'].between(1960,1978)]

In [ ]:
pre_1978_depreciation_df = penn_table_depreciation_df[penn_table_depreciation_df['year'] <= 1978]

### China

In [ ]:
china_GFCF_1960_1978_df = GFCF_1960_1978_df[GFCF_1960_1978_df['Country Code']=='CHN']

china_GFCF_growth_rate_1960_1978 = (china_GFCF_1960_1978_df['Value'] / china_GFCF_1960_1978_df['Value'].shift(1)-1).mean()

china_GFCF_1978 = china_GFCF_1960_1978_df[china_GFCF_1960_1978_df['Year']==1978]['Value']

china_1952_1978_depreciation_df = pre_1978_depreciation_df[pre_1978_depreciation_df['countrycode']=='CHN']['value'].mean()

china_capital_stock_1978 = (china_GFCF_1978/(china_GFCF_growth_rate_1960_1978 + china_1952_1978_depreciation_df)).values[0]

### India

In [ ]:
india_GFCF_1960_1978_df = GFCF_1960_1978_df[GFCF_1960_1978_df['Country Code']=='IND']

india_GFCF_growth_rate_1960_1978 = (india_GFCF_1960_1978_df['Value'] / india_GFCF_1960_1978_df['Value'].shift(1)-1).mean()

india_GFCF_1978 = india_GFCF_1960_1978_df[india_GFCF_1960_1978_df['Year']==1978]['Value']

india_1952_1978_depreciation_df = pre_1978_depreciation_df[pre_1978_depreciation_df['countrycode']=='IND']['value'].mean()

india_capital_stock_1978 = (india_GFCF_1978/(india_GFCF_growth_rate_1960_1978 + india_1952_1978_depreciation_df)).values[0]

## 1978-2024

In [ ]:
GFCF_1978_2024_df = GFCF_df[GFCF_df['Year'] >= 1978]

In [ ]:
post_1978_depreciation_df = penn_table_depreciation_df[penn_table_depreciation_df['year'] >= 1978]

In [ ]:
post_1978_depreciation_df = post_1978_depreciation_df[['country', 'year', 'value']]

In [ ]:
post_1978_depreciation_df.rename(columns={'country': 'Country Name'
                                         , 'year': 'Year'
                                         , 'value': 'Depreciation'}
                                      , inplace=True)

## China

In [ ]:
china_GFCF_1978_2024_df = GFCF_1978_2024_df[GFCF_1978_2024_df['Country Code']=='CHN']

china_GFCF_1978_2024_df = china_GFCF_1978_2024_df[['Country Name'
                                                   ,  'Year'
                                                   , 'Value'
                                                  ]]

china_GFCF_1978_2024_df.rename(columns={'Value': 'Gross fixed capital formation'
                                              }
                                      , inplace=True)

china_capital_stock_1978_2024_df = pd.merge(china_GFCF_1978_2024_df, post_1978_depreciation_df, how='left', on =['Country Name', 'Year'])

# ASSUMING THAT THE Depreciation RATE 2019-2024 is the same
china_capital_stock_1978_2024_df['Depreciation'] = china_capital_stock_1978_2024_df['Depreciation'].ffill()

china_capital_stock = [china_capital_stock_1978]
for index, row in china_capital_stock_1978_2024_df.iloc[1:].iterrows():
    china_capital_stock_current = china_capital_stock[-1] * (1 - row['Depreciation']) + row['Gross fixed capital formation']
    china_capital_stock.append(china_capital_stock_current)

china_capital_stock_1978_2024_df['Capital stock (current US$)'] = china_capital_stock

In [ ]:
china_capital_stock_1978_2024_df['Country Code'] = 'CHN'
china_capital_stock_1978_2024_df['Indicator Name'] = 'Capital stock (current US$)'
china_capital_stock_1978_2024_df['Indicator Code'] = 'CALC.CDM.CAPITAL.CD'
china_capital_stock_1978_2024_df['Source'] = 'Calculated by author'
china_capital_stock_1978_2024_df.rename(columns={'Capital stock (current US$)': 'Value'
                                                 }
                                      , inplace=True)
china_capital_stock_1978_2024_df = china_capital_stock_1978_2024_df[['Country Name'
                                                                     , 'Country Code'
                                                                     , 'Indicator Name'
                                                                     , 'Indicator Code'
                                                                     , 'Year'
                                                                     , 'Value'
                                                                     , 'Source'
                                                                    ]]

## India

In [ ]:
india_GFCF_1978_2024_df = GFCF_1978_2024_df[GFCF_1978_2024_df['Country Code']=='IND']

india_GFCF_1978_2024_df = india_GFCF_1978_2024_df[['Country Name'
                                                   ,  'Year'
                                                   , 'Value'
                                                  ]]

india_GFCF_1978_2024_df.rename(columns={'Value': 'Gross fixed capital formation'
                                              }
                                      , inplace=True)

india_capital_stock_1978_2024_df = pd.merge(india_GFCF_1978_2024_df, post_1978_depreciation_df, how='left', on =['Country Name', 'Year'])

# ASSUMING THAT THE Depreciation RATE 2019-2024 is the same
india_capital_stock_1978_2024_df['Depreciation'] = india_capital_stock_1978_2024_df['Depreciation'].ffill()

india_capital_stock = [india_capital_stock_1978]
for index, row in india_capital_stock_1978_2024_df.iloc[1:].iterrows():
    india_capital_stock_current = india_capital_stock[-1] * (1 - row['Depreciation']) + row['Gross fixed capital formation']
    india_capital_stock.append(india_capital_stock_current)

india_capital_stock_1978_2024_df['Capital stock (current US$)'] = india_capital_stock

In [ ]:
india_capital_stock_1978_2024_df['Country Code'] = 'IND'
india_capital_stock_1978_2024_df['Indicator Name'] = 'Capital stock (current US$)'
india_capital_stock_1978_2024_df['Indicator Code'] = 'CALC.CDM.CAPITAL.CD'
india_capital_stock_1978_2024_df['Source'] = 'Calculated by author'
india_capital_stock_1978_2024_df.rename(columns={'Capital stock (current US$)': 'Value'
                                                 }
                                      , inplace=True)
india_capital_stock_1978_2024_df = india_capital_stock_1978_2024_df[['Country Name'
                                                                     , 'Country Code'
                                                                     , 'Indicator Name'
                                                                     , 'Indicator Code'
                                                                     , 'Year'
                                                                     , 'Value'
                                                                     , 'Source'
                                                                    ]]

# Calculating the TFP

In [ ]:
cobb_doulgass_data_df = WDI_df[WDI_df['Indicator Code']!='NE.GDI.FTOT.CD']

In [ ]:
cobb_doulgass_data_df = pd.concat([cobb_doulgass_data_df, india_capital_stock_1978_2024_df, china_capital_stock_1978_2024_df])

In [ ]:
# cobb_doulgass_data_df = cobb_doulgass_data_df[cobb_doulgass_data_df['Year'] >= 1980]

In [ ]:
cobb_doulgass_data_df = cobb_doulgass_data_df[cobb_doulgass_data_df['Year'].between(1980,2023)]

## Function specification

In [ ]:
def total_factor_productivity_data_prep(data_df):
    # seperating them into the dependant (GDP) and independant (cap stock and labor) variables
    X = data_df[data_df.columns[1:-1]]
    y = data_df[data_df.columns[-1]]

    # taking the logs of the data so that the elasticity of the two independant variables would be their coefficicents
    X[X.columns[0]] = np.log(X[X.columns[0]].astype('float64'))
    X[X.columns[1]] = np.log(X[X.columns[1]].astype('float64'))
    
    # in a linearised version of cobb douglass (by taking the log of the equation) we get the log of TFP as a constant, in order to find it, we need to explicitly insert it
    X['TFP'] = 1
    
    # taking the log of the dependant variable (GDP) as per the lin version of cobb douglass
    y = np.log(y.astype('float64'))

    return X, y

### Constant model function

In [ ]:
# defining the MSE Function to minimize
def mse_loss(weights, X, y):
    # x_, x0, x1, x2 = x
    y_pred = X.dot(weights)
    return np.mean((y - y_pred) ** 2)

In [ ]:
def constant_cobb_douglass_model(data_df):

    # formatting the data
    X, y = total_factor_productivity_data_prep(data_df)


    # initial guess (all ones cause why not, it dont really matter)
    initial_guess = np.ones(len(X.T))*0.5
    
    # bounds (common bounds, elasticity and TFP are positive, but the last one doesnt have an upper bound 
    bounds = [(0, 1), (0, 1), (0, np.inf)]
    
    # constraints (assuming constant return to scale, thus the ealsticities must sum to 1)
    constraints = [
        {'type': 'eq'
         , 'fun': lambda weights: weights[0] + weights[1] - 1       # x1 + x2 = 1 (equality: x1 + x2 - 1 = 0)
        }
    ]
    
    # minimizing the mse function
    result = minimize(
        mse_loss,
        initial_guess,
        args=(X, y),
        method='trust-constr',
        bounds=bounds,
        constraints=constraints,
        options={'disp': True
                 , 'maxiter': 15000
                }
    )
    
    constant_elasticities_and_tfp_model_gdp = data_df['Capital stock (current US$)'] ** result.x[0] \
                                            * data_df['Labor force, total']** result.x[1] \
                                            * np.e ** result.x[2]
    
    constant_model_actual_mse = np.mean((constant_elasticities_and_tfp_model_gdp - data_df['GDP (current US$)']) ** 2)

    # Store results
    cobb_douglass_df = {'Capital stock elasticity (constant)': result.x[0]
                        , 'Labor force elasticity (constant)': result.x[1]
                        , 'Total factor productivity (constant)': np.e ** result.x[2]
                        # , 'Mean Squared Error': constant_model_actual_mse
                       }
        
    cobb_douglass_df = pd.DataFrame(data = np.array(list(cobb_douglass_df.values())).reshape(1, 3)
                                    , columns = cobb_douglass_df.keys()
                                   )
    
    return cobb_douglass_df, constant_elasticities_and_tfp_model_gdp

### Evolving model function 

In [ ]:
# defining the MSE Function to minimize
def mse_loss_window(weights, X_window, y_window):
    # x_, x0, x1, x2 = x
    y_pred = X_window.dot(weights)
    return np.mean((y_window - y_pred) ** 2)

In [ ]:
def evolving_cobb_douglass_model(data_df, window_size):

    # formatting the data
    X, y = total_factor_productivity_data_prep(data_df)
    
    results = []
    # window_size = 5
    
    for i in range(len(data_df['Year']) - window_size + 1):
        # Extract the current window
        X_window = X[i : i + window_size]
        y_window = y[i : i + window_size]
        
        # setting the initial guess (it doesnt really matter)
        if i == min(range(len(data_df['Year']) - window_size + 1)):
            initial_guess = np.ones(len(X.T))*0.5
        
        # bounds (common bounds, elasticity and TFP are positive, but the latter doesnt have an upper bound 
        bounds = [(0, 1), (0, 1), (0, np.inf)]
    
        # constraints (assuming constant return to scale, thus the ealsticities must sum to 1)
        constraints = [
            {'type': 'eq'
             , 'fun': lambda weights: weights[0] + weights[1] - 1       # x1 + x2 = 1 (equality: x1 + x2 - 1 = 0)
            }
        ]
        
        # minimizing the mse function
        result = minimize(
            mse_loss_window,
            initial_guess,
            args=(X_window, y_window),
            method='trust-constr',
            bounds=bounds,
            constraints=constraints,
            options={'disp': True
                     , 'maxiter': 15000
                    }
        )
    
        initial_guess = result.x
        
        evolving_elasticities_and_tfp_model_gdp = data_df['Capital stock (current US$)'] ** result.x[0] \
                                                * data_df['Labor force, total']** result.x[1] \
                                                * np.e ** result.x[2]
        
        evolving_model_actual_mse = np.mean((evolving_elasticities_and_tfp_model_gdp - data_df['GDP (current US$)']) ** 2)
    
        # Store results
        results.append({
            'Start': data_df['Year'][i],
            'Year': data_df['Year'][i + window_size - 1],
            'Capital stock elasticity (evolving)': result.x[0],
            'Labor force elasticity (evolving)': result.x[1],
            'Total factor productivity (evolving)': np.e ** result.x[2],
            # 'Mean Squared Error': evolving_model_actual_mse
        })
        
    cobb_douglass_df = pd.DataFrame(results)

    evolving_model_df = data_df[['Year','GDP (current US$)']]

    for i in range(len(data_df['Year']) - window_size + 1):
    
        Start = data_df['Year'][i]
        End = data_df['Year'][i + window_size - 1]
        
        five_year_sample_df = data_df[data_df['Year'].between(Start, End)]
        cobb_douglass = cobb_douglass_df[cobb_douglass_df['Start']==data_df['Year'][i]].reset_index(drop=True)
        five_year_sample_df[f'''GDP (current US$)_model_{End}'''] = five_year_sample_df['Capital stock (current US$)'] ** cobb_douglass['Capital stock elasticity (evolving)'][0] \
                                                                            * five_year_sample_df['Labor force, total'] ** cobb_douglass['Labor force elasticity (evolving)'][0] \
                                                                            * cobb_douglass['Total factor productivity (evolving)'][0]
        
        evolving_model_df = pd.merge(evolving_model_df, five_year_sample_df[['Year',f'''GDP (current US$)_model_{End}''']], how='left', on=['Year'])
    
    evolving_model_df['Evolving elasticities and TFP model GDP'] = evolving_model_df[evolving_model_df.columns[2:]].T.mean()
    evolving_elasticities_and_tfp_model_gdp = evolving_model_df['Evolving elasticities and TFP model GDP']
    
    return cobb_douglass_df, evolving_model_df, evolving_elasticities_and_tfp_model_gdp

## China

In [ ]:
china_cobb_doulgass_data_df = cobb_doulgass_data_df[cobb_doulgass_data_df['Country Code']=='CHN']

In [ ]:
china_cobb_doulgass_data_df = china_cobb_doulgass_data_df.pivot_table(index=['Year'],
                                                                      columns=['Indicator Name'],
                                                                      values=['Value'],
                                                                      aggfunc='sum')

In [ ]:
china_cobb_doulgass_data_df.columns = china_cobb_doulgass_data_df.columns.droplevel(0)

In [ ]:
china_cobb_doulgass_data_df.reset_index(drop=False
                                        , inplace=True)

In [ ]:
china_cobb_doulgass_data_df = china_cobb_doulgass_data_df[['Year'
                                                           , 'Capital stock (current US$)'
                                                           , 'Labor force, total'
                                                           , 'GDP (current US$)'
                                                          ]]

### constant TFP and elasticities

In [ ]:
china_constant_cobb_douglass_df, china_constant_model_gdp = constant_cobb_douglass_model(china_cobb_doulgass_data_df)

In [ ]:
china_constant_cobb_douglass_df = pd.concat([china_cobb_doulgass_data_df[['Year']], china_constant_cobb_douglass_df], axis = 1)

In [ ]:
china_constant_cobb_douglass_df['Capital stock elasticity (constant)'] = china_constant_cobb_douglass_df['Capital stock elasticity (constant)'].ffill()
china_constant_cobb_douglass_df['Labor force elasticity (constant)'] = china_constant_cobb_douglass_df['Labor force elasticity (constant)'].ffill()
china_constant_cobb_douglass_df['Total factor productivity (constant)'] = china_constant_cobb_douglass_df['Total factor productivity (constant)'].ffill()

In [ ]:
china_constant_cobb_douglass_df = pd.melt(china_constant_cobb_douglass_df, id_vars=list(china_constant_cobb_douglass_df.columns)[:1], value_vars=list(china_constant_cobb_douglass_df.columns)[1:])

In [ ]:
china_constant_cobb_douglass_df.rename(columns={'value': 'Value'
                                            , 'variable': 'Indicator Name'
                                           }
                                   , inplace=True)

In [ ]:
china_constant_cobb_douglass_df['Country Name'] = 'China'
china_constant_cobb_douglass_df['Country Code'] = 'CHN'
china_constant_cobb_douglass_df['Indicator Code'] = china_constant_cobb_douglass_df['Indicator Name'].apply(lambda x: 'CALC.CDM.CONST.CAPITAL.ELAST.CD' if x == 'Capital stock elasticity (constant)' 
                                                                                                    else 'CALC.CDM.CONST.LABOR.ELAST.CD' if x == 'Labor force elasticity (constant)'
                                                                                                    else 'CALC.CDM.CONST.TFP.CD')
china_constant_cobb_douglass_df['Source'] = 'Calculated by author'

In [ ]:
china_constant_cobb_douglass_df = china_constant_cobb_douglass_df[['Country Name'
                                                                   , 'Country Code'
                                                                   , 'Indicator Name'
                                                                   , 'Indicator Code'
                                                                   , 'Year'
                                                                   , 'Value'
                                                                   , 'Source']]

### evolving TFP and elasticities

In [ ]:
china_evolving_cobb_douglass_df, china_evolving_evolving_model_df, china_evolving_model_gdp = evolving_cobb_douglass_model(china_cobb_doulgass_data_df, 5)

In [ ]:
china_evolving_cobb_douglass_df.rename(columns={'End': 'Year'
                                                 }
                                      , inplace=True)

In [ ]:
china_evolving_cobb_douglass_df = china_evolving_cobb_douglass_df[['Year'
                                                                   , 'Capital stock elasticity (evolving)'
                                                                   , 'Labor force elasticity (evolving)'
                                                                   , 'Total factor productivity (evolving)'
                                                                  ]]

In [ ]:
china_evolving_cobb_douglass_df = pd.melt(china_evolving_cobb_douglass_df, id_vars=list(china_evolving_cobb_douglass_df.columns)[:1], value_vars=list(china_evolving_cobb_douglass_df.columns)[1:])

In [ ]:
china_evolving_cobb_douglass_df.rename(columns={'value': 'Value'
                                            , 'variable': 'Indicator Name'
                                           }
                                   , inplace=True)

In [ ]:
china_evolving_cobb_douglass_df['Country Name'] = 'China'
china_evolving_cobb_douglass_df['Country Code'] = 'CHN'
china_evolving_cobb_douglass_df['Indicator Code'] = china_evolving_cobb_douglass_df['Indicator Name'].apply(lambda x: 'CALC.CDM.EVOLV.CAPITAL.ELAST.CD' if x == 'Capital stock elasticity (evolving)'
                                                                                                    else 'CALC.CDM.EVOLV.LABOR.ELAST.CD' if x == 'Labor force elasticity (evolving)'
                                                                                                    else 'CALC.CDM.EVOLV.TFP.CD')
china_evolving_cobb_douglass_df['Source'] = 'Calculated by author'

In [ ]:
china_evolving_cobb_douglass_df = china_evolving_cobb_douglass_df[['Country Name'
                                                                   , 'Country Code'
                                                                   , 'Indicator Name'
                                                                   , 'Indicator Code'
                                                                   , 'Year'
                                                                   , 'Value'
                                                                   , 'Source']]

### Formatting

#### cobb_douglass

In [ ]:
china_cobb_douglass_df = pd.concat([china_constant_cobb_douglass_df, china_evolving_cobb_douglass_df]).reset_index(drop=True)

#### models

In [ ]:
china_cobb_doulgass_data_df['Evolving model GDP'] = china_evolving_model_gdp
china_cobb_doulgass_data_df['Constant model GDP'] = china_constant_model_gdp

In [ ]:
china_cobb_doulgass_data_df = china_cobb_doulgass_data_df[['Year'
                                                           , 'Evolving model GDP'
                                                           , 'Constant model GDP'
                                                          ]]

In [ ]:
china_cobb_doulgass_data_df = pd.melt(china_cobb_doulgass_data_df, id_vars=list(china_cobb_doulgass_data_df.columns)[:1], value_vars=list(china_cobb_doulgass_data_df.columns)[1:])

In [ ]:
china_cobb_doulgass_data_df.rename(columns={'value': 'Value'
                                           }
                                   , inplace=True)

In [ ]:
china_cobb_doulgass_data_df['Country Name'] = 'China'
china_cobb_doulgass_data_df['Country Code'] = 'CHN'
china_cobb_doulgass_data_df['Indicator Code'] = china_cobb_doulgass_data_df['Indicator Name'].apply(lambda x: 'CALC.CDM.EVOLV.GDP.CD' if x == 'Evolving model GDP'
                                                                                                    else 'CALC.CDM.CONST.GDP.CD')
china_cobb_doulgass_data_df['Source'] = 'Calculated by author'

In [ ]:
china_cobb_doulgass_data_df = china_cobb_doulgass_data_df[['Country Name'
                                                           , 'Country Code'
                                                           , 'Indicator Name'
                                                           , 'Indicator Code'
                                                           , 'Year'
                                                           , 'Value'
                                                           , 'Source']]

## India

In [ ]:
india_cobb_doulgass_data_df = cobb_doulgass_data_df[cobb_doulgass_data_df['Country Code']=='IND']

In [ ]:
india_cobb_doulgass_data_df = india_cobb_doulgass_data_df.pivot_table(index=['Year'],
                                                                      columns=['Indicator Name'],
                                                                      values=['Value'],
                                                                      aggfunc='sum')

In [ ]:
india_cobb_doulgass_data_df.columns = india_cobb_doulgass_data_df.columns.droplevel(0)

In [ ]:
india_cobb_doulgass_data_df.reset_index(drop=False
                                        , inplace=True)

In [ ]:
india_cobb_doulgass_data_df = india_cobb_doulgass_data_df[['Year'
                                                           , 'Capital stock (current US$)'
                                                           , 'Labor force, total'
                                                           , 'GDP (current US$)'
                                                          ]]

### constant TFP and elasticities

In [ ]:
india_constant_cobb_douglass_df, india_constant_model_gdp = constant_cobb_douglass_model(india_cobb_doulgass_data_df)

In [ ]:
india_constant_cobb_douglass_df = pd.concat([india_cobb_doulgass_data_df[['Year']], india_constant_cobb_douglass_df], axis = 1)

In [ ]:
india_constant_cobb_douglass_df['Capital stock elasticity (constant)'] = india_constant_cobb_douglass_df['Capital stock elasticity (constant)'].ffill()
india_constant_cobb_douglass_df['Labor force elasticity (constant)'] = india_constant_cobb_douglass_df['Labor force elasticity (constant)'].ffill()
india_constant_cobb_douglass_df['Total factor productivity (constant)'] = india_constant_cobb_douglass_df['Total factor productivity (constant)'].ffill()

In [ ]:
india_constant_cobb_douglass_df = pd.melt(india_constant_cobb_douglass_df, id_vars=list(india_constant_cobb_douglass_df.columns)[:1], value_vars=list(india_constant_cobb_douglass_df.columns)[1:])

In [ ]:
india_constant_cobb_douglass_df.rename(columns={'value': 'Value'
                                            , 'variable': 'Indicator Name'
                                           }
                                   , inplace=True)

In [ ]:
india_constant_cobb_douglass_df['Country Name'] = 'India'
india_constant_cobb_douglass_df['Country Code'] = 'IND'
india_constant_cobb_douglass_df['Indicator Code'] = india_constant_cobb_douglass_df['Indicator Name'].apply(lambda x: 'CALC.CDM.CONST.CAPITAL.ELAST.CD' if x == 'Capital stock elasticity (constant)' 
                                                                                                    else 'CALC.CDM.CONST.LABOR.ELAST.CD' if x == 'Labor force elasticity (constant)'
                                                                                                    else 'CALC.CDM.CONST.TFP.CD')
india_constant_cobb_douglass_df['Source'] = 'Calculated by author'

In [ ]:
india_constant_cobb_douglass_df = india_constant_cobb_douglass_df[['Country Name'
                                                                   , 'Country Code'
                                                                   , 'Indicator Name'
                                                                   , 'Indicator Code'
                                                                   , 'Year'
                                                                   , 'Value'
                                                                   , 'Source']]

### evolving TFP and elasticities

In [ ]:
india_evolving_cobb_douglass_df, india_evolving_evolving_model_df, india_evolving_model_gdp = evolving_cobb_douglass_model(india_cobb_doulgass_data_df, 5)

In [ ]:
india_evolving_cobb_douglass_df.rename(columns={'End': 'Year'
                                                 }
                                      , inplace=True)

In [ ]:
india_evolving_cobb_douglass_df = india_evolving_cobb_douglass_df[['Year'
                                                                   , 'Capital stock elasticity (evolving)'
                                                                   , 'Labor force elasticity (evolving)'
                                                                   , 'Total factor productivity (evolving)'
                                                                  ]]

In [ ]:
india_evolving_cobb_douglass_df = pd.melt(india_evolving_cobb_douglass_df, id_vars=list(india_evolving_cobb_douglass_df.columns)[:1], value_vars=list(india_evolving_cobb_douglass_df.columns)[1:])

In [ ]:
india_evolving_cobb_douglass_df.rename(columns={'value': 'Value'
                                            , 'variable': 'Indicator Name'
                                           }
                                   , inplace=True)

In [ ]:
india_evolving_cobb_douglass_df['Country Name'] = 'India'
india_evolving_cobb_douglass_df['Country Code'] = 'IND'
india_evolving_cobb_douglass_df['Indicator Code'] = india_evolving_cobb_douglass_df['Indicator Name'].apply(lambda x: 'CALC.CDM.EVOLV.CAPITAL.ELAST.CD' if x == 'Capital stock elasticity (evolving)'
                                                                                                    else 'CALC.CDM.EVOLV.LABOR.ELAST.CD' if x == 'Labor force elasticity (evolving)'
                                                                                                    else 'CALC.CDM.EVOLV.TFP.CD')
india_evolving_cobb_douglass_df['Source'] = 'Calculated by author'

In [ ]:
india_evolving_cobb_douglass_df = india_evolving_cobb_douglass_df[['Country Name'
                                                                   , 'Country Code'
                                                                   , 'Indicator Name'
                                                                   , 'Indicator Code'
                                                                   , 'Year'
                                                                   , 'Value'
                                                                   , 'Source']]

### Formatting

#### cobb_douglass

In [ ]:
india_cobb_douglass_df = pd.concat([india_constant_cobb_douglass_df, india_evolving_cobb_douglass_df]).reset_index(drop=True)

#### models

In [ ]:
india_cobb_doulgass_data_df['Evolving model GDP'] = india_evolving_model_gdp
india_cobb_doulgass_data_df['Constant model GDP'] = india_constant_model_gdp

In [ ]:
india_cobb_doulgass_data_df = india_cobb_doulgass_data_df[['Year'
                                                           , 'Evolving model GDP'
                                                           , 'Constant model GDP'
                                                          ]]

In [ ]:
india_cobb_doulgass_data_df = pd.melt(india_cobb_doulgass_data_df, id_vars=list(india_cobb_doulgass_data_df.columns)[:1], value_vars=list(india_cobb_doulgass_data_df.columns)[1:])

In [ ]:
india_cobb_doulgass_data_df.rename(columns={'value': 'Value'
                                           }
                                   , inplace=True)

In [ ]:
india_cobb_doulgass_data_df['Country Name'] = 'India'
india_cobb_doulgass_data_df['Country Code'] = 'IND'
india_cobb_doulgass_data_df['Indicator Code'] = india_cobb_doulgass_data_df['Indicator Name'].apply(lambda x: 'CALC.CDM.EVOLV.GDP.CD' if x == 'Evolving model GDP'
                                                                                                    else 'CALC.CDM.CONST.GDP.CD')
india_cobb_doulgass_data_df['Source'] = 'Calculated by author'

In [ ]:
india_cobb_doulgass_data_df = india_cobb_doulgass_data_df[['Country Name'
                                                           , 'Country Code'
                                                           , 'Indicator Name'
                                                           , 'Indicator Code'
                                                           , 'Year'
                                                           , 'Value'
                                                           , 'Source']]

# Stitiching and exporting

In [ ]:
pd.concat([india_cobb_doulgass_data_df
           , india_cobb_douglass_df
           , china_cobb_doulgass_data_df
           , china_cobb_douglass_df
           , cobb_doulgass_data_df]).to_excel(r'cobb_doulgass_calculation/cobb_doulgass_calculated_data.xlsx', index=False)